In [1]:
%pip install pandas
%pip install -r requirements.txt
%pip install optuna

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 25.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression, f_classif, SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [3]:
import zipfile

zipfile.ZipFile("./train.zip", "r").extractall()

In [4]:
# Загрузка данных (предположим, что данные в CSV)
df = pd.read_csv("./train.csv") # для train

In [5]:

# Удаляем индекс, так как он не несёт информации для модели
X = df.drop(['index', 'target'], axis=1)  # Все фичи (feature_1 ... feature_1366)
y = df['target']  # Целевая переменная

X, y

(        feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
 0        0.128367        1.0        1.0        1.0        1.0        4.0   
 1        0.258685        1.0        1.0        1.0        0.0        7.0   
 2        0.407738        1.0        1.0        1.0        2.0        7.0   
 3        0.439134        1.0        1.0        1.0        1.0        2.0   
 4        0.590481        1.0        1.0        1.0        0.0        1.0   
 ...           ...        ...        ...        ...        ...        ...   
 177853   0.660324        1.0        1.0        1.0        0.0        1.0   
 177854   0.222214        0.0        1.0        1.0        1.0        4.0   
 177855   0.545399        0.0        1.0        1.0        0.0        2.0   
 177856   0.393150        1.0        1.0        1.0        0.0        1.0   
 177857   0.395272        1.0        1.0        1.0        1.0        2.0   
 
         feature_6  feature_7  feature_8  feature_9  ...  feature_1357  \


In [6]:
indexes = df['index']
del df

In [7]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=200)),
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

X_selected = pipe.fit_transform(X, y)
selector = pipe.named_steps['kbest']

print(f"Исходное число признаков: {X.shape[1]}")
print(f"Число признаков после отбора: {X_selected.shape[1]}")

Исходное число признаков: 1367
Число признаков после отбора: 200


In [8]:
# 2. Масштабирование данных (важно для линейной регрессии)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42
)


In [ ]:
# 4. Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# 5. Предсказание и оценка качества
y_pred = model.predict(X_test)
# y_pred_proba_lin_reg = model.decision_function(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
# нормализуем значения для roc_auc
y_proba = 1 / (1 + np.exp(-y_pred)) # преобразование через сигмойду
roc_auc = roc_auc_score(y_test, y_proba)

print(f"\nMSE: {mse:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

# (Опционально) Вывод коэффициентов модели
if X_selected.shape[1] <= 20:  # Не выводить для слишком большого числа фич
    print("\nКоэффициенты модели:")
    # for feature, coef in zip(X.columns[selector.get_support()], model.coef_):
    #     print(f"{feature}: {coef:.4f}")
else:
    print(f"\nМодель имеет {X_selected.shape[1]} коэффициентов (слишком много для вывода).")


MSE: 0.0171
R2 Score: 0.0016
ROC-AUC Score: 0.6536

Модель имеет 400 коэффициентов (слишком много для вывода).


In [ ]:
y_pred

array([ 0.01629677,  0.00887584,  0.00911322, ...,  0.0150569 ,
       -0.0037384 , -0.00022041], shape=(53358,))

Так.... Из полученных метрик следует что вроде бы по MSE результат довольно терпимый и среднеквадратичная ошибка не слишком большая.
Касаемо метрики ROC-AUC Score получаем 0.6536 что является лучше чем убычное угадывание(подбрасывание монетки).
Из инересного мы не должны использовать LinearRegression, так как она может выдавать значения за пределами [0, 1], но после регуляризации значений, они могут быть использованиы для вычисления метрики.

Попробуем RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)


In [ ]:
y_pred_proba

array([0.  , 0.04, 0.03, ..., 0.  , 0.03, 0.  ], shape=(53358,))

In [ ]:
# generate output data
X_true_test = pd.read_csv('./datasets/test/test.csv')
X_true_test

,index,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_1357,feature_1358,feature_1359,feature_1360,feature_1361,feature_1362,feature_1363,feature_1364,feature_1365,feature_1366
0,124365,0.719921,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,177140,0.123122,0.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,106632,0.384612,0.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0,1,0,0,0,0,0
3,17630,0.452527,0.0,1.0,1.0,1.0,4.0,5.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,97189,0.379611,1.0,1.0,1.0,2.0,11.0,3.0,4.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76220,55074,0.325234,0.0,1.0,1.0,1.0,3.0,1.0,0.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76221,12771,0.227776,1.0,1.0,1.0,1.0,3.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76222,80999,0.459919,1.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
76223,31517,0.667716,1.0,1.0,1.0,0.0,2.0,2.0,2.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [ ]:
print(f"ROC_AUC_SCORE is {roc_auc:.4f}")

ROC_AUC_SCORE is 0.6261


Прогресс, модель достигла случайного угадывания
(последние данные ROC_AUC_SCORE is 0.6261, это даже лучше чем просто уадывание по среднему значению)

In [ ]:
mse = mean_squared_error(y_test, y_pred_proba)
print(f'MSE: {mse:.2f}')

MSE: 0.02


Небольшой апдейт для структуры модели

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)


In [ ]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6221


А что если заменить RandomForsetClassifier на LogisticRegression

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', LogisticRegression())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [ ]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6473


Вау, метрика подросла)


Теперь попробуем GradientBoostingClassifier


In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', GradientBoostingClassifier())
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [ ]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.6796


Попробуем еще одну модель SVC

In [ ]:
# 1. Отбор признаков: удаляем низковариативные (константные и почти константные)
# Сделаем pipeline для более эффективного отбора признаков(их сокращения)
pipe = Pipeline([
    ('selector', VarianceThreshold(threshold=0.01)),
    ('kbest', SelectKBest(f_classif, k=100)),
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True, kernel='rbf', C=1.0, decision_function_shape='ovr'))
    # ('lasso_selection', SelectFromModel(LassoCV(cv=2), max_features=500))
])

pipe.fit(X_train, y_train)

y_pred = pipe.predict_proba(X_test)[:, 1]

roc_auc_metric = roc_auc_score(y_test, y_pred)

In [ ]:
print(f"ROC-AUC Score is {roc_auc_metric:.4f}")

ROC-AUC Score is 0.5873


In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
    }

    model = XGBClassifier(**params, objective='binary:logistic')
    model.fit(X_train, y_train)
    y_proba = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_proba)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print(f"Лучший ROC-AUC: {study.best_value:.4f}")
print(f"Лучшие параметры: {study.best_params}")

[I 2025-04-22 14:11:51,119] A new study created in memory with name: no-name-87b7b70b-aafd-4135-9fa3-1bdcd706db98
[I 2025-04-22 14:17:52,409] Trial 0 finished with value: 0.6866250649470229 and parameters: {'n_estimators': 490, 'max_depth': 10, 'learning_rate': 0.16961501533881454, 'subsample': 0.966792220290188, 'colsample_bytree': 0.8259464128607417, 'reg_alpha': 0.8667944573109898, 'reg_lambda': 3.0138910295009715}. Best is trial 0 with value: 0.6866250649470229.
[I 2025-04-22 14:23:44,881] Trial 1 finished with value: 0.7080460523719639 and parameters: {'n_estimators': 756, 'max_depth': 5, 'learning_rate': 0.12189197148428516, 'subsample': 0.9825347727706664, 'colsample_bytree': 0.9920568553802339, 'reg_alpha': 6.3367269344870145, 'reg_lambda': 2.322292722370314}. Best is trial 1 with value: 0.7080460523719639.
[I 2025-04-22 14:30:54,791] Trial 2 finished with value: 0.7020038314632974 and parameters: {'n_estimators': 819, 'max_depth': 10, 'learning_rate': 0.15278706460806102, 'sub